## Imports and Device Setup

In [1]:
#Imports and Device Setup
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import numpy as np
import pandas as pd
import random
import time
import datetime
from tqdm import tqdm_notebook as tqdm
import gensim #For LDA
import gensim.corpora as corpora #For LDA
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

tf.config.experimental.list_physical_devices()

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

#Training Loop Imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


Now hooking up google drive to retrieve data files.

If you are trying to replicate this on your end, you may retrieve the dataset csv files and manually load them. Below are the links for those:
- Nepal Dataset: https://drive.google.com/open?id=1uYgcnTEZ5YEAdB6rLdWhHa7YuYJh9Wz3 
- Queensland Dataset: https://drive.google.com/open?id=1IbqkpjHk_lzqUgnPxAjPifTbECmZ3p1P 
- Irony Dataset: https://drive.google.com/file/d/1yCULJV5EDc_EoHcNTxP8ovZ903KxJtYV/view?usp=sharing 
- Hyperpartisan Bias Dataset: https://drive.google.com/open?id=1k4cSuS1Ww2U92CnEckVIh_QqF3JUkIsO 

In [3]:
# I am choosing to mount my google drive which contains the datasets shown above
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Installing Hugging Face Library for pretrained BERT 

In [2]:
!pip install transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup


In [3]:
#PATHS TO DATASETS
nepal_data_path = '/content/drive/My Drive/Academics/Grad/Spring_20/NLP_Project/preprocessed_nepal.csv'
queensland_data_path = '/content/drive/My Drive/Academics/Grad/Spring_20/NLP_Project/preprocessed_queensland.csv'
irony_data_path = '/content/drive/My Drive/Academics/Grad/Spring_20/NLP_Project/trainA.csv'
bias_data_path = '/content/drive/My Drive/Academics/Grad/Spring_20/NLP_Project/bias_only.csv'

import multiprocessing
multiprocessing.cpu_count()

4

## Helper Functions for BERT Model 
- Splitting Text by Topics
- BERT Tokenization
- BERT Padding
- BERT Attention Masking
- Evaluation Metrics for during training

In [0]:
def texts_labels(df, by_topic = False, label = 'label', content = 'text', upsample = False):
  """ Returning lists of texts and labels. Topic based option available""" 
  n = len(df)
  if not by_topic:
    return (df[content].values, df[label].values)
  else:
    n_topics = df.topic.max()+1
    data = []
    for tpc in range(n_topics):
      _df = df[df.topic == tpc]
      nn = len(_df)
      if upsample:
        _df = df.sample(frac = n/nn, replace=True).reset_index(drop=True)
      juicy_stuff = (_df[content].values, _df[label].values)
      data.append(juicy_stuff)
    return data

def _get_input_ids(sentences, tokenizer):
  """Tokenizes all of the sentences and map the tokens to thier word IDs."""

  input_ids = []

  # For every sentence...
  for sent in tqdm(sentences, desc='Running Tokenizer'):
      # `encode` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      encoded_sent = tokenizer.encode(
                          sent,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'

                          # This function also supports truncation and conversion
                          # to pytorch tensors, but we need to do padding, so we
                          # can't use these features :( .
                          #max_length = 128,          # Truncate all sentences.
                          #return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.
      input_ids.append(encoded_sent)
  
  return input_ids


def _pad_tokens(input_ids,tokenizer,MAX_LEN=64):
  """This function pads tokens to the specified MAX_LENGTH"""
  # We'll borrow the `pad_sequences` utility function to do this.
  # Set the maximum sequence length.
  # I've chosen 64 somewhat arbitrarily. It's slightly larger than the

  print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

  print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

  # Pad our input tokens with value 0.
  # "post" indicates that we want to pad and truncate at the end of the sequence,
  # as opposed to the beginning.
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                            value=0, truncating="post", padding="post")

  print('\nDone.')
  return input_ids

def _mask(input_ids):
  """This function creates attention masks"""
  attention_masks = []

  # For each sentence...
  for sent in tqdm(input_ids,desc='Masking'):
      
      # Create the attention mask.
      #   - If a token ID is 0, then it's padding, set the mask to 0.
      #   - If a token ID is > 0, then it's a real token, set the mask to 1.
      att_mask = [int(token_id > 0) for token_id in sent]
      
      # Store the attention mask for this sentence.
      attention_masks.append(att_mask)
  
  return attention_masks


Evaluation Metric Helper Functions

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_f1(preds,labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(pred_flat,labels_flat,average="macro")

def num_correct(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat), len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
    
#Helper Functions to Evaluate Topic Based BERT Finetuning
def naive_mean(accss):
  accs_mean = []
  accs_std = []
  for topic in accss.keys():
    accs = []
    for run in accss[topic].keys():
      accs.append(max(accss[topic][run]))
    
    accs_mean.append(np.mean(accs))
    accs_std.append(np.std(accs))

  return np.mean(accs_mean), np.mean(accs_std)

def topic_based_acc(tc, tt, num_topics):
  num_topics += 1
  n_runs = max(tc[1].keys()) + 1
  run_accuracies = []
  for i in range(n_runs):
    max_tc = [max(tc[topic][i]) for topic in range(1,num_topics)]
    _tt = [tt[topic][i][0] for topic in range(1,num_topics)]
    sum_tc = np.sum(max_tc)
    sum_tt = np.sum(_tt)
    run_accuracies.append(sum_tc/sum_tt)
  return np.mean(run_accuracies), np.std(run_accuracies)


def _result_helper(correct, total):
  accs = []
  for i in range(len(correct)):
    accs.append(max(correct[i])/total[i][0])
  
  return np.mean(accs),np.std(accs)

def topic_based_lr(fl_lr, acc_lr, tt):
	topics = fl_lr.keys()
	f1_wt_sum = 0
	acc_wt_sum = 0
	n_total = 0
	for topic in topics:
		n = tt[topic][0][0]
		f1_wt_sum += (fl_lr[topic]*n)
		acc_wt_sum += (acc_lr[topic]*n)
		n_total += n

	weighted_avg_f1 = f1_wt_sum/n_total
	weighted_avg_acc = acc_wt_sum/n_total

	return weighted_avg_f1, weighted_avg_acc

def just_f1(f1, tt):
  topics = f1.keys()
  f1_wt_sum = 0
  acc_wt_sum = 0
  n_total = 0
  for topic in topics:
    n = tt[topic][0][0]
    f1_wt_sum += (f1[topic]*n)
    n_total += n

  weighted_avg_f1 = f1_wt_sum/n_total

  return weighted_avg_f1


In [0]:
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

## Topic Modeling (LDA)

Here, I will build the various function needed to run LDA for use in the BERT fine-tuning pipeline. 

In [0]:
def _assign_topics(df, id2word, lda_model, content):
  "This function uses the lda model to assign topics to examples"
  from operator import itemgetter
  topics = []
  for i in range(len(df)):
      text = df.loc[i, content].split()
      text = id2word.doc2bow(text)

      topic_probs = lda_model[text]
      if (len(topic_probs) == 0):
          # If text is too short, assign random topic.
          topic = np.random.choice(int(num_topics))

          while (not os.path.isfile(word2vec_path + '_' + str(topic) + '_wv.model')):
              topic = np.random.choice(int(num_topics))
      else:
          # Compute most prevalant topic.
          topic = max(topic_probs, key=itemgetter(1))[0]

      topics.append(topic)

  df['topic'] = topics

  return df

def run_LDA(csv_path, label, content, n, multi = False, POS=0, n_samples=0):
  """Takes the provided path to csv dataset and perfoms topic modeling
    Returns DataFrame with an assigned 'topic' column.
  """
  print("..Running LDA for %d topics"%n)

  df = pd.read_csv(csv_path)
  if n_samples == 0:
    df = df.sample(frac=1)
    df = df.reset_index(drop=True)
  else:
    df = df.sample(n=n_samples)
    df = df.reset_index(drop=True)
  df_ = df.copy()
  if POS==1:
    df[content] = df[content].apply(nouns)
  if POS==2:
    df[content] = df[content].apply(nouns_adj)

  text = df[content]

  texts = []

  for sentence in text:
    words = sentence.split(' ')
    texts.append(words)

  id2word = corpora.Dictionary(texts)

  corpus = [id2word.doc2bow(text) for text in texts]

  if n>1:
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=n, random_state=100, update_every=1, passes=50)
    df = _assign_topics(df, id2word, lda_model, content)
  else:
    df['topic'] = 0

  print(df.topic.value_counts())
  df_.topic = df.topic
  print("..LDA ~~fin")

  return df_


## Training Loop (BERT Finetuning)

In [0]:
def train_loop(run, input_ids, labels, attention_masks, params, verbose=False):
  """MAIN TRAINING LOOP. Loads Pretrained Bert and performs finetuning"""

  # Use 90% for training and 10% for validation.
  train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                              random_state=params['SEED'], test_size=params['TEST_SIZE'])
  # Do the same for the masks.
  train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                              random_state=int(run*10), test_size=params['TEST_SIZE'])

  train_inputs = torch.tensor(train_inputs)
  validation_inputs = torch.tensor(validation_inputs)

  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(validation_labels)

  train_masks = torch.tensor(train_masks)
  validation_masks = torch.tensor(validation_masks)


  batch_size = params['BATCH_SIZE']

  # Create the DataLoader for our training set.
  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  # Create the DataLoader for our validation set.
  validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  model = BertForSequenceClassification.from_pretrained(
      "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
      num_labels = params['NUM_CLASSES'], # The number of output labels--2 for binary classification. You can increase this for multi-class tasks.  
      #hidden_dropout_prob = params['DROPOUT'],
      output_attentions = False, # Whether the model returns attentions weights.
      output_hidden_states = True, # Whether the model returns all hidden-states.
  )

  # Tell pytorch to run this model on the GPU.
  model.cuda()
  
  optimizer = AdamW(model.parameters(),
                lr = params['LEARNING_RATE'], # args.learning_rate - default is 5e-5, our notebook had 2e-5
                eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
              )
  
  epochs = params['EPOCHS']

  # Total number of training steps is number of batches * number of epochs.
  total_steps = len(train_dataloader) * epochs

  # Create the learning rate scheduler.
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)
  
  

  
  # Set the seed value all over the place to make this reproducible.
  seed_val = params['SEED']
  #
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # Store the average loss after each epoch so we can plot them.
  loss_values = []
  validation_acc = []

  total_correct = []
  total_total = []

  r = run+1
  print("______TRAINING RUN #%d______"%r)
  
  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.
      if verbose:
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_loss = 0

      # Put the model into training mode. 
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              if verbose:
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # This will return the loss (rather than the model output) 
          outputs = model(b_input_ids, 
                      token_type_ids=None, 
                      attention_mask=b_input_mask, 
                      labels=b_labels)
          
          # The call to `model` always returns a tuple, so we need to pull the 
          # loss value out of the tuple.
          loss = outputs[0]

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()


      # Calculate the average loss over the training data.
      avg_train_loss = total_loss / len(train_dataloader)            
      
      # Store the loss value for plotting the learning curve.
      loss_values.append(avg_train_loss)

      if verbose:
        print("")
        print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
      
      
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.
      
      print("Running Validation for Epoch %d..."%epoch_i)

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      eval_loss, eval_accuracy, eval_f1 = 0, 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      nb_total_correct = 0
      nb_total_total = 0
      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Add batch to GPU
          batch = tuple(t.to(device) for t in batch)
          
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          
          # Telling the model not to compute or store gradients, saving memory and
          # speeding up validation
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              outputs = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask)
          
          # Get the "logits" output by the model. The "logits" are the output
          # values prior to applying an activation function like the softmax.
          logits = outputs[0]

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          
          # Calculate the metrics for this batch of test sentences.
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          correct_ex, total_ex = num_correct(logits, label_ids)
          tmp_eval_f1 = flat_f1(logits, label_ids)

          nb_total_correct += correct_ex
          nb_total_total += total_ex
          
          # Accumulate the total accuracy and F1.
          eval_accuracy += tmp_eval_accuracy
          eval_f1 += tmp_eval_f1

          # Track the number of batches
          nb_eval_steps += 1

      # Report the final accuracy for this validation run.
      accu = eval_accuracy/nb_eval_steps
      f1 = eval_f1/nb_eval_steps
      print(" Val. Accuracy: {0:.4f}. ({1:4d}/{2:4d})".format(accu,nb_total_correct,nb_total_total))
      print(" Val. F1 Score: {0:.4f}.)".format(f1))
      total_correct.append(nb_total_correct)
      total_total.append(nb_total_total)
      validation_acc.append(accu)
      if verbose:
        print("  Validation took: {:}".format(format_time(time.time() - t0)))
  
  return loss_values, validation_acc, total_correct, total_total, f1, model

TESTING ON LOGISTIC REGRESSION

In [0]:
def test_LR(run, input_ids, attention_masks, labels, model, params):

  # Convert to tensors.
  prediction_inputs = torch.tensor(input_ids)
  prediction_masks = torch.tensor(attention_masks)
  prediction_labels = torch.tensor(labels)

  # Set the batch size.  
  batch_size = params['BATCH_SIZE']  

  # Create the DataLoader.
  prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  print('Extracting Contextual Embeddings for {:,} test sentences...'.format(len(prediction_inputs)))

  # Put model in evaluation mode
  model.eval()

  # Tracking variables 
  embeddings , true_labels = [], []

  embedded_train = np.zeros((df.shape[0],params['MAX_LEN'],768))
  testing_labs = np.zeros((df.shape[0],))

  # Predict 
  for batch in tqdm(prediction_dataloader, desc='Extracting Contextual Embeddings'):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)
        #outputs = model(b_input_ids)

    ######logits = outputs[1][0][0]
    logits = outputs[1][0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # Store predictions and true labels
    ######embeddings.append(logits)
    ######true_labels.append(label_ids)
    #pdb.set_trace()
    #print("DEBUG >> Logits shape: ",logits.shape)
    for i in range(batch_size):
      #print("DEBUG >> Logits shape: ",logits.shape)
      try:
        embeddings.append(logits[i])
        true_labels.append(label_ids[i])
      except:
        break

  for i in range(len(embeddings)):
    embedded_train[i,:,:] = embeddings[i]
    testing_labs[i] = true_labels[i]

  
  X = embedded_train.sum(axis=1)
  Y = testing_labs
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=params['SEED'], test_size=params['TEST_SIZE'])
  seed = 7
  model = LogisticRegression()
  model.fit(X_train, Y_train)

  y_pred = model.predict(X_test)

  f1 = f1_score(Y_test, y_pred, average="macro")
  acc = accuracy_score(Y_test, y_pred)
  #precision_score(y_test, y_pred, average="macro"))
  #print(recall_score(y_test, y_pred, average="macro"))  

  print("Logistic Regression Accuracy: %.4f"%acc)
  print("Logistic Regression F1 score: %.4f"%f1)

  return f1, acc


In [0]:
def test_LR(run, input_ids, attention_masks, labels, model, params):

  return 0,0

### Topic Based Finetuning 

In [0]:
def topic_based_BERT(data, params):
  """ Runs Training loop for finetuning one topic at a time. 
      Returns Evaluation Metrics for each topic """

  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
  
  results = {}
  tp_losses = {}
  tp_validation_acc = {}
  tp_total_correct = {}
  tp_total_total = {}
  tp_f1 = {}

  tp_lr_f1 = {}
  tp_lr_acc = {}

  i = 1
  n = len(data)
  for topic_set in data:
    print("============================================")
    print("Fine-Tuning BERT for Topic set {0:2d}/{1:2d}".format(i,n))
    print("============================================")
    print("")
    sentences, labels = topic_set
    input_ids = _get_input_ids(sentences, tokenizer)
    input_ids = _pad_tokens(input_ids, tokenizer, MAX_LEN=params['MAX_LEN'])
    attention_masks = _mask(input_ids)

    print("~~~~~")
    print("Running Training Loop %d times"%params['N_RUNS'])

    r_losses = {}
    r_validation_acc = {}
    r_total_correct = {}
    r_total_total = {}
    r_f1 = {}

    for run in range(params['N_RUNS']):
      run_loss, run_val_acc, run_correct, run_total, run_f1, model = train_loop(run,input_ids,labels,attention_masks, params, verbose = False)
      r_losses[run] = run_loss
      r_validation_acc[run] = run_val_acc
      r_total_correct[run] = run_correct
      r_total_total[run] = run_total
      r_f1 = run_f1

      """VALIDATING ON LOGISTIC REGRESSION AS WELL"""
      r_lr_f1,r_lr_acc = test_LR(run, input_ids, attention_masks, labels, model, params)
      

    tp_losses[i] = r_losses
    tp_validation_acc[i] = r_validation_acc
    tp_total_correct[i] = r_total_correct
    tp_total_total[i] = r_total_total
    tp_f1[i] = r_f1

    #Logistic Regression Metrics
    tp_lr_f1[i] = r_lr_f1
    tp_lr_acc[i] = r_lr_acc

    ovr_acc_mean, ovr_acc_std  = _result_helper(r_total_correct, r_total_total)
    results[i] = {'acc_mean': ovr_acc_mean, 'acc_std': ovr_acc_std}

    print("~~~~~")
    print("Training Complete for topic set {0:2d}/{1:2d}".format(i,n))

    i+=1
  
  return tp_losses, tp_validation_acc, tp_total_correct, tp_total_total, results, tp_lr_f1, tp_lr_acc, tp_f1

## Results

- Varying Number of Topics [1,2,4,8]. Note that 1 represents topic independent finetuning
- Multiple Finetuning Runs per Topic (define this in the training parameters)
- Keep an eye on training_param 
- Validation set performance is output for both BERT's default classification head (Feed forward network with dropout = 0.1) and a Logistic Regression classifer.
- Results are printed out at the end of the run as a dictionary.

*Please note that the Logistic Regression is giving abnormally high performance, as a result this was not included in the report. The Feed Forward network's validation accuracies and F1 scores was used instead.*

### Nepal Dataset

From previous observation, max token length = 63. Hence we will set MAX_LEN = 64. 

In [16]:
import pdb
print("######    STARTING TOPIC BASED FINETUNING    ######")
results = {}
for num_topics in [1,2,4,8]:
    print("******************NUM_TOPICS: %d******************" % num_topics)
    print("")

    df = run_LDA(
        nepal_data_path, label="relevance_label", content="text", n=num_topics
    )
    print("Returned this many topics: ", df.topic.max() + 1)
    assert (df.topic.max() + 1) == num_topics

    data = texts_labels(
        df, by_topic=True, label="relevance_label", content="text"
    )
    t0 = time.time()
    training_params = {
        "N_RUNS": 1,
        "TEST_SIZE": 0.1,
        "MAX_LEN": 64,
        "BATCH_SIZE": 32,
        "NUM_CLASSES": 2,
        "LEARNING_RATE": 5e-5,
        "EPOCHS": 2,
        "SEED": 42,
        "DROPOUT": 0.1,
    }
    lossess, accss, tc, tt, rslts, f1_lr, acc_lr, f_1 = topic_based_BERT(data, training_params)
    overall_acc_mean, val_acc_std = topic_based_acc(tc, tt, num_topics)
    f1_mean = just_f1(f_1, tt)
    #pdb.set_trace()
    lr_f1_mean , lr_acc_mean = topic_based_lr(f1_lr, acc_lr, tt)
    naive_acc_mean, naive_acc_std = naive_mean(accss)
    assert len(accss.keys()) == num_topics
    print("******************START SUMMARY******************")
    print("Number of Topics: %d"%num_topics)
    print(
        "Val Acc (Mean per topic): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            naive_acc_mean, naive_acc_std
        )
    )
    print(
        "Val Acc (#TotalCorrect/#TotalExamples): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            overall_acc_mean, val_acc_std
        )
    )
    print(
        "Val F1 (Weighted Avg.): {0:.4f})".format(
            f1_mean
        )
    )
    print(
        "LR Val Acc (Weighted Avg.): {0:.4f})".format(
            lr_acc_mean
        )
    )
    print(
        "LR Val F1 (Weighted Avg.): {0:.4f})".format(
            lr_f1_mean
        )
    )
    print("Time Taken: {:}".format(format_time(time.time() - t0)))
    print("******************END SUMMARY******************")
    print("***********************************************")
    results[num_topics] = {
        "Val_Acc_MEAN": overall_acc_mean,
        "Val_Acc_STD": val_acc_std,
        "Val_f1": f1_mean,
        "Naive_Acc_MEAN": naive_acc_mean,
        "Naive_Acc_STD": naive_acc_std,
        "LR_Acc": lr_acc_mean,
        "LR_F1": lr_f1_mean
    }


######    STARTING TOPIC BASED FINETUNING    ######
******************NUM_TOPICS: 1******************

..Running LDA for 1 topics
0    8449
Name: topic, dtype: int64
..LDA ~~fin
Returned this many topics:  1
Fine-Tuning BERT for Topic set  1/ 1





Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.54
Running Validation for Epoch 0...
 Val. Accuracy: 0.7559. ( 637/ 845)
 Val. F1 Score: 0.7444.)
  Average training loss: 0.42
Running Validation for Epoch 1...
 Val. Accuracy: 0.7664. ( 649/ 845)
 Val. F1 Score: 0.7588.)
~~~~~
Training Complete for topic set  1/ 1
******************START SUMMARY******************
Number of Topics: 1
Val Acc (Mean per topic): (MEAN: 0.7664 // STD: 0.000)
Val Acc (#TotalCorrect/#TotalExamples): (MEAN: 0.7680 // STD: 0.000)
Val F1 (Weighted Avg.): 0.7588)
LR Val Acc (Weighted Avg.): 0.0000)
LR Val F1 (Weighted Avg.): 0.0000)
Time Taken: 0:01:48
******************END SUMMARY******************
***********************************************
******************NUM_TOPICS: 2******************

..Running LDA for 2 topics
1    4293
0    4156
Name: topic, dtype: int64
..LDA ~~fin
Returned this many topics:  2
Fine-Tuning BERT for Topic set  1/ 2





Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.55
Running Validation for Epoch 0...
 Val. Accuracy: 0.7668. ( 319/ 416)
 Val. F1 Score: 0.7611.)
  Average training loss: 0.42
Running Validation for Epoch 1...
 Val. Accuracy: 0.7596. ( 316/ 416)
 Val. F1 Score: 0.7556.)
~~~~~
Training Complete for topic set  1/ 2
Fine-Tuning BERT for Topic set  2/ 2



...


~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.68
Running Validation for Epoch 0...
 Val. Accuracy: 0.7500. (   9/  12)
 Val. F1 Score: 0.7483.)
  Average training loss: 0.64
Running Validation for Epoch 1...
 Val. Accuracy: 0.8333. (  10/  12)
 Val. F1 Score: 0.8286.)
~~~~~
Training Complete for topic set  6/ 8
Fine-Tuning BERT for Topic set  7/ 8





Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.60
Running Validation for Epoch 0...
 Val. Accuracy: 0.7857. (  11/  14)
 Val. F1 Score: 0.4400.)
  Average training loss: 0.54
Running Validation for Epoch 1...
 Val. Accuracy: 0.7857. (  11/  14)
 Val. F1 Score: 0.4400.)
~~~~~
Training Complete for topic set  7/ 8
Fine-Tuning BERT for Topic set  8/ 8





Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.59
Running Validation for Epoch 0...
 Val. Accuracy: 0.7125. ( 105/ 148)
 Val. F1 Score: 0.4146.)
  Average training loss: 0.54
Running Validation for Epoch 1...
 Val. Accuracy: 0.7600. ( 112/ 148)
 Val. F1 Score: 0.6200.)
~~~~~
Training Complete for topic set  8/ 8
******************START SUMMARY******************
Number of Topics: 8
Val Acc (Mean per topic): (MEAN: 0.7710 // STD: 0.000)
Val Acc (#TotalCorrect/#TotalExamples): (MEAN: 0.7774 // STD: 0.000)
Val F1 (Weighted Avg.): 0.7068)
LR Val Acc (Weighted Avg.): 0.0000)
LR Val F1 (Weighted Avg.): 0.0000)
Time Taken: 0:02:14
******************END SUMMARY******************
***********************************************


In [17]:
import pprint
pprint.pprint(results)

{1: {'LR_Acc': 0.0,
     'LR_F1': 0.0,
     'Naive_Acc_MEAN': 0.7663817663817665,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.7680473372781065,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.758765279732276},
 2: {'LR_Acc': 0.0,
     'LR_F1': 0.0,
     'Naive_Acc_MEAN': 0.7574568288854004,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.7565011820330969,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.7465107843165952},
 4: {'LR_Acc': 0.0,
     'LR_F1': 0.0,
     'Naive_Acc_MEAN': 0.7450644841269841,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.7671394799054374,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.7161298162766128},
 8: {'LR_Acc': 0.0,
     'LR_F1': 0.0,
     'Naive_Acc_MEAN': 0.7709553380647131,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.7773851590106007,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.7067655854063988}}


### Queensland dataset

In [35]:
import pdb
print("######    STARTING TOPIC BASED FINETUNING    ######")
results = {}
for num_topics in [1,2,4,8]:
    print("******************NUM_TOPICS: %d******************" % num_topics)
    print("")

    df = run_LDA(
        queensland_data_path, label="relevance_label", content="text", n=num_topics
    )
    print("Returned this many topics: ", df.topic.max() + 1)
    assert (df.topic.max() + 1) == num_topics

    data = texts_labels(
        df, by_topic=True, label="relevance_label", content="text"
    )
    t0 = time.time()
    training_params = {
        "N_RUNS": 1,
        "TEST_SIZE": 0.1,
        "MAX_LEN": 64,
        "NUM_CLASSES": 2,
        "BATCH_SIZE": 32,
        "LEARNING_RATE": 5e-5,
        "EPOCHS": 2,
        "SEED": 42,
        "DROPOUT": 0.1,
    }
    lossess, accss, tc, tt, rslts, f1_lr, acc_lr, f_1 = topic_based_BERT(data, training_params)
    overall_acc_mean, val_acc_std = topic_based_acc(tc, tt, num_topics)
    f1_mean = just_f1(f_1, tt)
    #pdb.set_trace()
    lr_f1_mean , lr_acc_mean = topic_based_lr(f1_lr, acc_lr, tt)
    naive_acc_mean, naive_acc_std = naive_mean(accss)
    assert len(accss.keys()) == num_topics
    print("******************START SUMMARY******************")
    print("Number of Topics: %d"%num_topics)
    print(
        "Val Acc (Mean per topic): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            naive_acc_mean, naive_acc_std
        )
    )
    print(
        "Val Acc (#TotalCorrect/#TotalExamples): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            overall_acc_mean, val_acc_std
        )
    )
    print(
        "Val F1 (Weighted Avg.): {0:.4f})".format(
            f1_mean
        )
    )
    print(
        "LR Val Acc (Weighted Avg.): {0:.4f})".format(
            lr_acc_mean
        )
    )
    print(
        "LR Val F1 (Weighted Avg.): {0:.4f})".format(
            lr_f1_mean
        )
    )
    print("Time Taken: {:}".format(format_time(time.time() - t0)))
    print("******************END SUMMARY******************")
    print("***********************************************")
    results[num_topics] = {
        "Val_Acc_MEAN": overall_acc_mean,
        "Val_Acc_STD": val_acc_std,
        "Val_f1": f1_mean,
        "Naive_Acc_MEAN": naive_acc_mean,
        "Naive_Acc_STD": naive_acc_std,
        "LR_Acc": lr_acc_mean,
        "LR_F1": lr_f1_mean
    }


######    STARTING TOPIC BASED FINETUNING    ######
******************NUM_TOPICS: 1******************

..Running LDA for 1 topics
0    7062
Name: topic, dtype: int64
..LDA ~~fin
Returned this many topics:  1
Fine-Tuning BERT for Topic set  1/ 1





Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.20
Running Validation for Epoch 0...
 Val. Accuracy: 0.9497. ( 670/ 707)
 Val. F1 Score: 0.9487.)
  Average training loss: 0.10
Running Validation for Epoch 1...
 Val. Accuracy: 0.9443. ( 666/ 707)
 Val. F1 Score: 0.9431.)
Extracting Contextual Embeddings for 7,062 test sentences...



Logistic Regression Accuracy: 0.9590
Logistic Regression F1 score: 0.9589
~~~~~
Training Complete for topic set  1/ 1
******************START SUMMARY******************
Number of Topics: 1
Val Acc (Mean per topic): (MEAN: 0.9497 // STD: 0.000)
Val Acc (#TotalCorrect/#TotalExamples): (MEAN: 0.9477 // STD: 0.000)
Val F1 (Weighted Avg.): 0.9431)
LR Val Acc (Weighted Avg.): 0.9590)
LR Val F1 (Weighted Avg.): 0.9589)
Time Taken: 0:01:46
******************END SUMMARY******************
***********************************************
******************NUM_TOPICS: 2******************

..Running LDA for 2 topics
1    3824
0    3238
Name: topic, dtype: int64
..LDA ~~fin
Returned this many topics:  2
Fine-Tuning BERT for Topic set  1/ 2





Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.


...


~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.62
Running Validation for Epoch 0...
 Val. Accuracy: 0.8333. (  15/  18)
 Val. F1 Score: 0.8036.)
  Average training loss: 0.48
Running Validation for Epoch 1...
 Val. Accuracy: 0.8889. (  16/  18)
 Val. F1 Score: 0.8750.)
Extracting Contextual Embeddings for 178 test sentences...



Logistic Regression Accuracy: 0.9986
Logistic Regression F1 score: 0.9824
~~~~~
Training Complete for topic set  7/ 8
Fine-Tuning BERT for Topic set  8/ 8





Padding/truncating all sentences to 64 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.29
Running Validation for Epoch 0...
 Val. Accuracy: 0.9766. ( 110/ 113)
 Val. F1 Score: 0.7440.)
  Average training loss: 0.16
Running Validation for Epoch 1...
 Val. Accuracy: 0.9766. ( 110/ 113)
 Val. F1 Score: 0.7440.)
Extracting Contextual Embeddings for 1,123 test sentences...



Logistic Regression Accuracy: 0.9929
Logistic Regression F1 score: 0.9862
~~~~~
Training Complete for topic set  8/ 8
******************START SUMMARY******************
Number of Topics: 8
Val Acc (Mean per topic): (MEAN: 0.9587 // STD: 0.000)
Val Acc (#TotalCorrect/#TotalExamples): (MEAN: 0.9577 // STD: 0.000)
Val F1 (Weighted Avg.): 0.8933)
LR Val Acc (Weighted Avg.): 0.9877)
LR Val F1 (Weighted Avg.): 0.9539)
Time Taken: 0:02:19
******************END SUMMARY******************
***********************************************


In [36]:
pprint.pprint(results)

{1: {'LR_Acc': 0.958981612446959,
     'LR_F1': 0.9589342904583263,
     'Naive_Acc_MEAN': 0.9497282608695652,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.9476661951909476,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.9430976904923386},
 2: {'LR_Acc': 0.9705311003923185,
     'LR_F1': 0.9254920697040063,
     'Naive_Acc_MEAN': 0.9588755498533725,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.9561527581329562,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.9020391847119152},
 4: {'LR_Acc': 0.9849167725489256,
     'LR_F1': 0.9386123715284901,
     'Naive_Acc_MEAN': 0.958319655987395,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.9590395480225988,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.8578269355742978},
 8: {'LR_Acc': 0.9876984680359384,
     'LR_F1': 0.9539241800204149,
     'Naive_Acc_MEAN': 0.95869213570845,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.9577464788732394,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.8932907027579529}}


### Irony Dataset

In [40]:
import pdb
print("######    STARTING TOPIC BASED FINETUNING    ######")
results = {}
for num_topics in [1,2,4,8]:
    print("******************NUM_TOPICS: %d******************" % num_topics)
    print("")

    df = run_LDA(
        irony_data_path, label="label", content="text", n=num_topics
    )
    print("Returned this many topics: ", df.topic.max() + 1)
    assert (df.topic.max() + 1) == num_topics

    data = texts_labels(
        df, by_topic=True, label="label", content="text"
    )
    t0 = time.time()
    training_params = {
        "N_RUNS": 1,
        "TEST_SIZE": 0.1,
        "MAX_LEN": 42,
        "NUM_CLASSES": 2,
        "BATCH_SIZE": 32,
        "LEARNING_RATE": 5e-5,
        "EPOCHS": 2,
        "SEED": 42,
        "DROPOUT": 0.1,
    }
    lossess, accss, tc, tt, rslts, f1_lr, acc_lr, f_1 = topic_based_BERT(data, training_params)
    overall_acc_mean, val_acc_std = topic_based_acc(tc, tt, num_topics)
    f1_mean = just_f1(f_1, tt)
    #pdb.set_trace()
    lr_f1_mean , lr_acc_mean = topic_based_lr(f1_lr, acc_lr, tt)
    naive_acc_mean, naive_acc_std = naive_mean(accss)
    assert len(accss.keys()) == num_topics
    print("******************START SUMMARY******************")
    print("Number of Topics: %d"%num_topics)
    print(
        "Val Acc (Mean per topic): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            naive_acc_mean, naive_acc_std
        )
    )
    print(
        "Val Acc (#TotalCorrect/#TotalExamples): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            overall_acc_mean, val_acc_std
        )
    )
    print(
        "Val F1 (Weighted Avg.): {0:.4f})".format(
            f1_mean
        )
    )
    print(
        "LR Val Acc (Weighted Avg.): {0:.4f})".format(
            lr_acc_mean
        )
    )
    print(
        "LR Val F1 (Weighted Avg.): {0:.4f})".format(
            lr_f1_mean
        )
    )
    print("Time Taken: {:}".format(format_time(time.time() - t0)))
    print("******************END SUMMARY******************")
    print("***********************************************")
    results[num_topics] = {
        "Val_Acc_MEAN": overall_acc_mean,
        "Val_Acc_STD": val_acc_std,
        "Val_f1": f1_mean,
        "Naive_Acc_MEAN": naive_acc_mean,
        "Naive_Acc_STD": naive_acc_std,
        "LR_Acc": lr_acc_mean,
        "LR_F1": lr_f1_mean
    }


######    STARTING TOPIC BASED FINETUNING    ######
******************NUM_TOPICS: 1******************

..Running LDA for 1 topics
0    3834
Name: topic, dtype: int64
..LDA ~~fin
Returned this many topics:  1
Fine-Tuning BERT for Topic set  1/ 1





Padding/truncating all sentences to 42 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.69
Running Validation for Epoch 0...
 Val. Accuracy: 0.5573. ( 214/ 384)
 Val. F1 Score: 0.5466.)
  Average training loss: 0.64
Running Validation for Epoch 1...
 Val. Accuracy: 0.6120. ( 235/ 384)
 Val. F1 Score: 0.5978.)
Extracting Contextual Embeddings for 3,834 test sentences...



Logistic Regression Accuracy: 0.6042
Logistic Regression F1 score: 0.6014
~~~~~
Training Complete for topic set  1/ 1
******************START SUMMARY******************
Number of Topics: 1
Val Acc (Mean per topic): (MEAN: 0.6120 // STD: 0.000)
Val Acc (#TotalCorrect/#TotalExamples): (MEAN: 0.6120 // STD: 0.000)
Val F1 (Weighted Avg.): 0.5978)
LR Val Acc (Weighted Avg.): 0.6042)
LR Val F1 (Weighted Avg.): 0.6014)
Time Taken: 0:00:44
******************END SUMMARY******************
***********************************************
******************NUM_TOPICS: 2******************

..Running LDA for 2 topics
1    2296
0    1538
Name: topic, dtype: int64
..LDA ~~fin
Returned this many topics:  2
Fine-Tuning BERT for Topic set  1/ 2





Padding/truncating all sentences to 42 values...

Padding token: "[PAD]", ID: 0

Done.


...


~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.71
Running Validation for Epoch 0...
 Val. Accuracy: 0.3125. (  13/  36)
 Val. F1 Score: 0.2364.)
  Average training loss: 0.68
Running Validation for Epoch 1...
 Val. Accuracy: 0.3438. (  15/  36)
 Val. F1 Score: 0.2522.)
Extracting Contextual Embeddings for 360 test sentences...



Logistic Regression Accuracy: 0.9401
Logistic Regression F1 score: 0.7493
~~~~~
Training Complete for topic set  7/ 8
Fine-Tuning BERT for Topic set  8/ 8





Padding/truncating all sentences to 42 values...

Padding token: "[PAD]", ID: 0

Done.



~~~~~
Running Training Loop 1 times
______TRAINING RUN #1______
  Average training loss: 0.69
Running Validation for Epoch 0...
 Val. Accuracy: 0.6000. (  18/  30)
 Val. F1 Score: 0.3750.)
  Average training loss: 0.68
Running Validation for Epoch 1...
 Val. Accuracy: 0.6333. (  19/  30)
 Val. F1 Score: 0.4599.)
Extracting Contextual Embeddings for 296 test sentences...



Logistic Regression Accuracy: 0.9479
Logistic Regression F1 score: 0.7481
~~~~~
Training Complete for topic set  8/ 8
******************START SUMMARY******************
Number of Topics: 8
Val Acc (Mean per topic): (MEAN: 0.5587 // STD: 0.000)
Val Acc (#TotalCorrect/#TotalExamples): (MEAN: 0.5633 // STD: 0.000)
Val F1 (Weighted Avg.): 0.4285)
LR Val Acc (Weighted Avg.): 0.9189)
LR Val F1 (Weighted Avg.): 0.7542)
Time Taken: 0:01:14
******************END SUMMARY******************
***********************************************


In [41]:
pprint.pprint(results)

{1: {'LR_Acc': 0.6041666666666666,
     'LR_F1': 0.6013986013986015,
     'Naive_Acc_MEAN': 0.6119791666666666,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.6119791666666666,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.5978447679241402},
 2: {'LR_Acc': 0.7735595703125,
     'LR_F1': 0.7091589122225225,
     'Naive_Acc_MEAN': 0.5851462339743589,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.5807291666666666,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.5323031142275806},
 4: {'LR_Acc': 0.8586647727272727,
     'LR_F1': 0.7522314252958283,
     'Naive_Acc_MEAN': 0.6353546626984127,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.5974025974025974,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.5458783663661924},
 8: {'LR_Acc': 0.9188872739018088,
     'LR_F1': 0.7541827382482452,
     'Naive_Acc_MEAN': 0.558732793381535,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.5633074935400517,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.4284744594739497}}


### Hyperpartisan Bias Dataset

This dataset is a big boy. Not only is it a lot of examples, but each example is a news article consisting of many sentences. 

From previous exploration of this dataset, the following are statistics on  token lengths of news articles in this dataset:



```
print('Max sentence length: ', max([len(sen) for sen in input_ids]))
print('Avg sentence length: ', np.mean([len(sen) for sen in input_ids]))
```
```
Max sentence length:  18644
Avg sentence length:  853.4613
```
The max token length of BERT is 512. As a compromise, we will only be able to use the first portion of each article to make our prediction.
To be reasonable with training time, we will take only the first 10,000 examples as well.


In [0]:
import pdb
print("######    STARTING TOPIC BASED FINETUNING    ######")
results = {}
for num_topics in [1,2,4]:
    print("******************NUM_TOPICS: %d******************" % num_topics)
    print("")

    df = run_LDA(
        bias_data_path, label="hyperpartisan", content="content", n=num_topics,
        n_samples = 10000
    )
    print("Returned this many topics: ", df.topic.max() + 1)
    assert (df.topic.max() + 1) == num_topics

    data = texts_labels(
        df, by_topic=True, label="hyperpartisan", content="content"
    )
    t0 = time.time()
    training_params = {
        "N_RUNS": 1,
        "TEST_SIZE": 0.1,
        "MAX_LEN": 512,
        "BATCH_SIZE": 8,
        "NUM_CLASSES": 2,
        "LEARNING_RATE": 5e-5,
        "EPOCHS": 2,
        "SEED": 42,
        "DROPOUT": 0.1,
    }
    lossess, accss, tc, tt, rslts, f1_lr, acc_lr, f_1 = topic_based_BERT(data, training_params)
    overall_acc_mean, val_acc_std = topic_based_acc(tc, tt, num_topics)
    f1_mean = just_f1(f_1, tt)
    #pdb.set_trace()
    lr_f1_mean , lr_acc_mean = topic_based_lr(f1_lr, acc_lr, tt)
    naive_acc_mean, naive_acc_std = naive_mean(accss)
    assert len(accss.keys()) == num_topics
    print("******************START SUMMARY******************")
    print("Number of Topics: %d"%num_topics)
    print(
        "Val Acc (Mean per topic): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            naive_acc_mean, naive_acc_std
        )
    )
    print(
        "Val Acc (#TotalCorrect/#TotalExamples): (MEAN: {0:.4f} // STD: {1:.3f})".format(
            overall_acc_mean, val_acc_std
        )
    )
    print(
        "Val F1 (Weighted Avg.): {0:.4f})".format(
            f1_mean
        )
    )
    print(
        "LR Val Acc (Weighted Avg.): {0:.4f})".format(
            lr_acc_mean
        )
    )
    print(
        "LR Val F1 (Weighted Avg.): {0:.4f})".format(
            lr_f1_mean
        )
    )
    print("Time Taken: {:}".format(format_time(time.time() - t0)))
    print("******************END SUMMARY******************")
    print("***********************************************")
    results[num_topics] = {
        "Val_Acc_MEAN": overall_acc_mean,
        "Val_Acc_STD": val_acc_std,
        "Val_f1": f1_mean,
        "Naive_Acc_MEAN": naive_acc_mean,
        "Naive_Acc_STD": naive_acc_std
        #"LR_Acc": lr_acc_mean,
        #"LR_F1": lr_f1_mean
    }


In [15]:
import pprint
pprint.pprint(results)

{1: {'Naive_Acc_MEAN': 0.957,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.957,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.9126400710400714},
 2: {'Naive_Acc_MEAN': 0.944683908045977,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.939,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.8736949571255939},
 4: {'Naive_Acc_MEAN': 0.9409205533220912,
     'Naive_Acc_STD': 0.0,
     'Val_Acc_MEAN': 0.9401197604790419,
     'Val_Acc_STD': 0.0,
     'Val_f1': 0.8488370252566512}}


## Discussion of Results


- TI = Topic-independent
- TD = Topic-dependent (using LDA)

| Dataset | TI Accuracy | TD Accuracy| TI F-1 | TD F-1 |
|---------|-------------|------------|--------|--------|
|Nepal disaster | 74.96% | <font color=red>75.77%</font> | 74.26% | <font color=red>75.09%</font> |
|Queensland disaster | 94.97% | <font color=red>95.62%</font> | 94.31% | 90.20% |
 |Irony classification | 61.20% | 59.74% | 59.78% | 54.58% |
 |Bias classification | 95.70% | 94.47% | 91.26% | 87.37% |




From the results above, we notice an overall high Accuracy and F-1 performance using BERT (compared to other baselines that can be found in our report). <br>

Topic based fine-tuning of BERT seemed to have made a slight improvement initially based on the Nepal disaster Dataset. However, this same improvement was not evident for the 3 other datasets. There could be two reasons for this: (i) The semantic and syntactic information that are needed are different for each task. Topic based finetuning may have hindered the BERT model from capturing some of that information for all datasets except the first one. (ii) The nepal dataset is an anomaly. 

Another potential confounding factor that may have been due to smaller amount of data available for fine-tuning once a dataset was split up into topics. 
BERT seems to already be doing a good job disambiguating word senses to start with. Eg. whether we break it into topics or not, its going to be able to distinguish between "apple" the laptop and "apple" the fruit . By breaking up the dataset into topics, we are just providing it with less data to fine-tune on. 

